<a href="https://colab.research.google.com/github/vibhumkhanna/My_projects/blob/main/Brax_env.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brax Environments

Brax provides [Gym](https://gym.openai.com/)-like environments for training policies.  Just as in Gym, Brax environments have step and reset functions that return agent observations and rewards.  Let's try some out!

In [ ]:
#@title Import Brax and some helper modules

import functools
import time

from IPython.display import HTML, Image 
import gym

try:
  import brax
except ImportError:
  from IPython.display import clear_output 
  !pip install git+https://github.com/google/brax.git@main
  clear_output()
  import brax

from brax import envs
from brax import jumpy as jp
from brax.envs import to_torch
from brax.io import html
from brax.io import image
import jax
from jax import numpy as jnp
import torch
v = torch.ones(1, device='cuda')  # init torch cuda before jax

RuntimeError: ignored

In [ ]:
#@title Visualizing pre-included Brax environments { run: "auto" }
#@markdown Select an environment to preview it below:

environment = "humanoid"  # @param ['ant', 'halfcheetah', 'hopper', 'humanoid', 'reacher', 'walker2d', 'fetch', 'grasp', 'ur5e']
env = envs.create(env_name=environment)
state = env.reset(rng=jp.random_prngkey(seed=0))

HTML(html.render(env.sys, [state.qp]))

# Stepping and Jitting Environments

We operate environments via `env.step`:

$$
\text{state}_{t+1} = \text{env.step}(\text{state}_t, \text{action})
$$


The state returned by `env.step` contains:
* the physics state `QP`
* the observation vector
* reward
* whether the environment is done and needs to be reset
* other useful metrics.

By calling `env.step` in sequence we produce a rollout:

In [ ]:
%%time
rollout = []
for i in range(100):
  # wiggle sinusoidally
  action = jp.ones((env.action_size,)) * jp.sin(i * jp.pi / 15)
  state = env.step(state, action)
  rollout.append(state)

CPU times: user 29.5 s, sys: 210 ms, total: 29.7 s
Wall time: 30.6 s


In order to train policies quickly, we use [JAX](https://github.com/google/jax) to jit compile the `env.step`: 

In [ ]:
# jit compile env.step:
state = jax.jit(env.step)(state, jnp.ones((env.action_size,)))

After `env.step` has been compiled, it runs much faster:

In [ ]:
%%time
for _ in range(100):
  state = jax.jit(env.step)(state, jnp.ones((env.action_size,)))

CPU times: user 302 ms, sys: 35.5 ms, total: 338 ms
Wall time: 316 ms


# Visualizing Rollouts

Brax provides two handy ways to visualize environment rollouts.  The first is an HTML visualizer:

In [ ]:
HTML(html.render(env.sys, [s.qp for s in rollout]))

The second is an image visualizer that produces animated GIFs or PNGs:

In [ ]:
Image(image.render(env.sys, [s.qp for s in rollout], width=320, height=240))

NameError: ignored

# OpenAI Gym Wrapper + PyTorch

Brax environments can converted to Gym environments, using either [GymWrapper](https://github.com/google/brax/blob/main/brax/envs/wrappers.py) for single environments, or [VectorGymWrapper](https://github.com/google/brax/blob/main/brax/envs/wrappers.py) for batched (parallelized) environments.  Gym-wrapped Brax environments are jit-compiled, and can interoperate with (and share accelerator resources with) other learning frameworks such as PyTorch.

In [ ]:
entry_point = functools.partial(envs.create_gym_env, env_name='ant')
if 'brax-ant-v0' not in gym.envs.registry.env_specs:
  gym.register('brax-ant-v0', entry_point=entry_point)

# create a gym environment that contains 4096 parallel ant environments
gym_env = gym.make("brax-ant-v0", batch_size=4096)

# wrap it to interoperate with torch data structures
gym_env = to_torch.JaxToTorchWrapper(gym_env, device='cuda')

# jit compile env.reset
obs = gym_env.reset()

# jit compile env.step
action = torch.rand(gym_env.action_space.shape, device='cuda') * 2 - 1
obs, reward, done, info = gym_env.step(action)

Here, we generate a rollout using our Gym wrapper with PyTorch data:

In [ ]:
before = time.time()

for _ in range(100):
  action = torch.rand(gym_env.action_space.shape, device='cuda') * 2 - 1
  obs, rewards, done, info = gym_env.step(action)

duration = time.time() - before
print(f'time for {409600} steps: {duration:.2f}s ({int(409600 / duration)} steps/sec)')

time for 409600 steps: 1.59s (257295 steps/sec)
